In [78]:
from flatsurf import polygons
from flatsurf import MutableOrientedSimilaritySurface as MOSS
from sage.combinat.permutation import Permutation  # , SymmetricGroup
from sage.all import SymmetricGroup


def all_transpositions(n):
    perms = []
    for i in range(1, n+1):
        for j in range(i, n+1):
            if i != j:
                perms.append((i, j))
    return perm


def all_transpositions_of_elements(elements):
    perms = []
    n = len(elements)

    for i in range(n):
        for j in range(i, n):
            if i != j:
                perms.append((elements[i], elements[j]))

    return perms

# this is an example of a for loop that goes through a list of numbers


def all_pairs(list_1, list_2):
    pairs = []
    for i in range(len(list_1)):
        for j in range(len(list_2)):
            pairs.append((list_1[i], list_2[j]))

    return pairs


def compute_valid_perms(elements):
    n = len(elements)
    if n % 2 == 1:
        raise ValueError(
            'Can only compute valid permutations for an even number of elements.')

    # TODO: gotta be a better way of doing this
    evens = []
    odds = []
    for e in elements:
        if e % 2 == 0:
            evens.append(e)
        if e % 2 == 1:
            odds.append(e)

    if len(evens) != len(odds):
        raise ValueError('Number of even and odd elements were different.')

    if n == 2:
        return [[(odds[0], evens[0])]]

    valid_perms = []

    for i, e in enumerate(evens):  # enumerate gives you both the element and the index
        first_perm = (odds[0], e)
        other_odds = odds[1:]
        other_evens = evens[:i] + evens[i+1:]
        other_elements = other_evens + other_odds
        other_perms = compute_valid_perms(other_elements)

        for perm in other_perms:
            valid_perms.append([first_perm]+perm)

    return valid_perms

# pair is a list of tuples
# want to convert it to a Permutation type from the sage SymmetricGroup library
# e.g. pair = [[(1,2),(3,4)], [(1,3), (2,4)]]
# return (Permutation('(12)(34)'), Permutation('(14)(23)))


def convert_pair(n, pair):
    Sn = SymmetricGroup(n)
    top = Sn(pair[0])
    bottom = Sn(pair[1])
    return (top, bottom)


# we dont want there to be the same pairings ie cannot have T and B (14) and (14), dont wanna include
# goal is to output a list of pairs of permutations
# two permutations create a pair and check if they share any permutations, if they do share discard, dont.. keep
def compute_valid_pairs(n, valid_perms):
    valid_pairings = []
    for i, v in enumerate(valid_perms):
        for j, w in enumerate(valid_perms):
            first_pair = (v, w)
            # print(f"v is {v}") #this is to check whats going on in the for loop. f is a formatted print combines text w variable v
            # print(f"w is {w}")
            # enumerate for index and element of list
            if check_if_perms_share_transposition(v, w):
                # print("they're equal")
                continue  # use continue to say "dont include"
            if j < i:
                continue
            else:
                # print(f"they're not equal, appending: {first_pair}")
                pair = convert_pair(n, first_pair)
                valid_pairings.append(pair)

    return valid_pairings


def check_if_perms_share_transposition(perm1, perm2):
    # e.g. perm1 = [(1, 2), (3, 4), (5, 6)]
    #      perm2 = [(1, 4), (3, 2), (5, 6)]

    for t1 in perm1:
        for t2 in perm2:
            if t1 == t2:
                return True

    return False


# This function takes in two inputs:
# - n which is the number of dots (permutation group)
# - bottom which is the bottom permutation
# It returns a permutation which is the shift of bottom

def shift_bottom(n, bottom):
    Sn = SymmetricGroup(n)
    s = ''
    for i in range(1, n+1):
        if i == 1:
            s = s+'(1,'
        elif i < n:
            s = s + f'{n-i+2},'
        if i == n:
            s = s + '2)'

    si = ''
    for i in range(1, n+1):
        if i == 1:
            si = si+'(1,'
        elif i < n:
            si = si + f'{i},'
        if i == n:
            si = si + f'{n})'

    s = Sn(s)
    si = Sn(si)
    new = s*bottom*si  # TODO: is this correct? ... i believe it is
    return new


# This is the first n=8 example, the list of valid perms is way too long. WHat i need to do is create a function that takes this list of valid perms at n =8 and checks the order every time i shift the perm
# we already defined the shifting function. i think i need to define an order function

# i expect perm to be (top,bottom) where top= Sn('(1,2)(3,4)(5,6)') and bottom= Sn('(1,4)(3,6)(2,5)')
def permutation_order(n, perm):
    (top, bottom) = perm
    # print(top)
    # print(bottom)
    before_shift = (top*bottom).order()
    # print(before_shift)

    old_bottom = bottom
    for i in range(n):  # this will make the for loop happen n times even tho were not using i
        new_bottom = shift_bottom(n, old_bottom)
        order = (top*new_bottom).order()
        if order == n/2:
            old_bottom = new_bottom
            continue
        else:
            return False  # this is how we say NO
    return True


def check_all_perms(n, valid_perms):
    perm_order = []
    # p is a variab;le that represents an element of the list "elements"..[a,b,c] first loop=a second loop=b
    for p in valid_perms:

        if permutation_order(n, p) == True:
            # a dot means calling a function , p is the variable that is equaling each valid perm thats why it goes there
            perm_order.append(p)
        else:
            continue
    return perm_order  # return your new list


# i need something that calculates the order after the shifts

# This function takes in two inputs:
# - n which is the number of dots (permutation group)
# - bottom which is the bottom permutation
# It returns a permutation which is the shift of bottom
def shift_bottom(n, bottom):
    Sn = SymmetricGroup(n)
    s = ''
    for i in range(1, n+1):
        if i == 1:
            s = s+'(1,'
        elif i < n:
            s = s + f'{n-i+2},'
        if i == n:
            s = s + '2)'

    si = ''
    for i in range(1, n+1):
        if i == 1:
            si = si+'(1,'
        elif i < n:
            si = si + f'{i},'
        if i == n:
            si = si + f'{n})'

    s = Sn(s)
    si = Sn(si)
    new = s*bottom*si  # TODO: is this correct? ... i believe it is
    return new


# now we're going to write a bespoke initializer for our class of PCTS

# n is the number of squares
# Top and bottom should be permutations of the form:
# top = SymmetricGroup(4)('(1, 3)(2,4)')
# - disjoint transpositions
# - every number present

def build_pillowcase(n, top, bottom):
    P = MOSS(AA)

    for i in range(1, n+1):
        P.add_polygon(polygons.square(), label=i)

    for i in range(1, n+1):
        if i < n:
            P.glue((i, 1), (i+1, 3))

        P.glue((i, 0), (top(i), 0))
        P.glue((i, 2), (bottom(i), 2))

    P.glue((1, 3), (n, 1))
    # P.glue((n,1), (n,1))

    P.set_immutable()
    P.category()
    return P


In [68]:
from flatsurf import translation_surfaces

# the library has some nice built in examples
S = translation_surfaces.square_torus()
S = translation_surfaces.octagon_and_squares()
S.set_immutable()

# we can investigate the translation surface with the following methods
S.category()
S.stratum() # Note: calling stratum() only works after calling category() (otherwise flatsurf hasn't yet computed what type of surface this is, so it  doesn't know how to check the stratum: https://flatsurf.github.io/sage-flatsurf/geometry/categories.html#module-flatsurf.geometry.categories)
S.category().axioms()
S.is_translation_surface()
S.genus()
S.components()
S.gluings()
S.is_connected()
S.opposite_edge((0,0))

(1, 0)

In [69]:
# Building a bespoke translation surface
# https://flatsurf.github.io/sage-flatsurf/geometry/surface.html#flatsurf.geometry.surface.MutableOrientedSimilaritySurface
from flatsurf import MutableOrientedSimilaritySurface as MOSS
# Question: MOSS vs MutablePolygonalSurface?

# Initialize the surface
surface = MOSS(AA) # The AA means use the algebraic reals. Other option is QQ for rationals. 
# Question: why choose one or the other?

# add as many polygons as you want, optionally specify the label (default is numbers starting at 0)
surface.add_polygon(polygons.square())

# then specify how to glue the squares together. the polygons have default edge ordering 0,1,2, ... 
# # Question: can these be specified?
surface.glue((0, 0), (0, 2))
surface.glue((0, 1), (0, 3))

# finish the bespoke craftmanship
surface.set_immutable()
surface.category()

# then you can go investigate it like in the previous cell

surface.genus()
surface.stratum()


H_1(0)

In [70]:
# build a translation surface using a permutation

p1 = SymmetricGroup(4)('(1, 3)(2,4)')
p2 = SymmetricGroup(4)('')
S = translation_surfaces.origami(p1, p2)
# S.plot()

In [71]:
# building a half-translation surface
# note: pillowcase tiled surfaces are the subset of half translation surfaces that are built from squares (just like square tiled surfaces are the subset of translation surfaces that are built from squares)
# https://flatsurf.github.io/sage-flatsurf/geometry/half_translation_surfaces.html#module-flatsurf.geometry.categories.half_translation_surfaces

P = MOSS(AA) 

P.add_polygon(polygons.square())
P.add_polygon(polygons.square())

P.glue((0, 1), (1, 3))
P.glue((0, 3), (1, 1))
P.glue((0, 2), (1, 2))
P.glue((0, 0), (1, 0))

P.set_immutable()
P.category()

# now you can investigate it with all the same functions as above
P.is_translation_surface() # should be: False
# P.genus()
# P.stratum()
# P.plot(edge_labels=True)


False

In [72]:
# now we're going to write a bespoke initializer for our class of PCTS
from flatsurf import MutableOrientedSimilaritySurface as MOSS
from flatsurf import polygons

# n is the number of squares
# Top and bottom should be permutations of the form:
# top = SymmetricGroup(4)('(1, 3)(2,4)')
# - disjoint transpositions
# - every number present 
def build_pillowcase(n, top, bottom):
    P = MOSS(AA)

    for i in range(1,n+1):
        P.add_polygon(polygons.square(), label=i)

    for i in range(1,n+1):
        if i < n:
            P.glue((i,1), (i+1, 3))

        P.glue((i,0), (top(i), 0))
        P.glue((i,2), (bottom(i), 2))
    
    P.glue((1,3), (n,1))
    # P.glue((n,1), (n,1))

    P.set_immutable()
    P.category()
    return P

n = 4
Sn = SymmetricGroup(n)
top = Sn('(1,2)(3,4)') 
bottom = Sn('(1,4)(2,3)')
pillowcase = build_pillowcase(n, top, bottom)
pillowcase.category()
pillowcase.genus()
pillowcase.stratum()


    




Q_0(0^2, -1^4)

In [73]:
# This function takes in two inputs:
# - n which is the number of dots (permutation group)
# - bottom which is the bottom permutation 
# It returns a permutation which is the shift of bottom
def shift_bottom(n, bottom):
    Sn = SymmetricGroup(n)
    s=''
    for i in range(1,n+1):    
        if i==1:
            s=s+'(1,'
        elif i<n : 
            s=s+ f'{n-i+2},'
        if i==n:
            s=s+ '2)'
      
    si= ''
    for i in range(1,n+1):
        if i==1:
            si= si+'(1,'
        elif i<n:
            si= si+ f'{i},'
        if i==n:
            si= si+ f'{n})'
   
    s = Sn(s)
    si = Sn(si)
    new = s*bottom*si # TODO: is this correct? ... i believe it is 
    return new

n = 4
Sn = SymmetricGroup(n)
top = Sn('(1,2)(3,4)') 
bottom = Sn('(1,4)(2,3)')
new_bottom = shift_bottom(n, bottom)
print(new_bottom)
shifted_pillowcase = build_pillowcase(n, top, new_bottom)
# # TODO: figure out how to compute cylinders

(1,2)(3,4)


In [74]:
#trying to play around 
n=4 
Sn = SymmetricGroup(n)
top = Sn('(1,2)(3,4)') 
bottom = Sn('(1,4)(2,3)')
(top*bottom).order()
new_bottom= shift_bottom(n,bottom)
top * new_bottom
(top*new_bottom).order() #order should be 1 

# new_bottom= shift_bottom(4,bottom)
# new_bottom.order()
# # pillowcase.genus()
# # pillowcase.stratum()






1

In [75]:
#computing the order of a group 
G= SymmetricGroup(2)
G
G.order()



2

In [76]:
valid_perms= compute_valid_perms(set)
compute_valid_pairs(valid_perms)
# okay so I think i need to define a function that computes the order of a par of permutations. 
from sage.combinat.permutation import Permutation

def permutation_order(valid_perms):

   
    return order 
    
    
# i would need something to define the pair i think?

def pair_order(perm1,perm2)
    
    order1 = permutation_order(perm1)
    order2 = permutation_order(perm2)
    

    return pair_order
    






SyntaxError: expected ':' (4034329044.py, line 14)

In [ ]:
# i used valid permutation pair from the list 
n=6 
Sn = SymmetricGroup(n) 
top = Sn('(1,2)(3,4)(5,6)') 
bottom = Sn('(1,4)(3,6)(2,5)')
pillowcase = build_pillowcase(n, top, bottom)
pillowcase.category()
new_bottom= shift_bottom(n,bottom)
top * new_bottom
(top*bottom).order()
(top*new_bottom).order()

new_bottom2= shift_bottom(n,new_bottom)
(top*new_bottom2).order() #has order 3 now 

new_bottom3= shift_bottom(n,new_bottom2)
new_bottom3
(top*new_bottom3).order() #has order 3

new_bottom4=shift_bottom(n,new_bottom3)
new_bottom4
(top*new_bottom4).order() #has order 3

new_bottom5=shift_bottom(n,new_bottom4)
new_bottom5
(top*new_bottom5).order() #order 3 






NameError: name 'SymmetricGroup' is not defined

In [ ]:
n=6 
Sn = SymmetricGroup(n)
top= Sn('(1,2)(3,4)(5,6)')
bottom= Sn('(1,6)(2,3)(4,5)')
top * bottom 
(top*bottom).order() #has order 3 

new_bottom= shift_bottom(n,bottom)
new_bottom
top * new_bottom #keeps returning an empty ()
(top*new_bottom).order() #has order 1

new_bottom2= shift_bottom(n,new_bottom)
new_bottom2
(top*new_bottom2).order() #has order 3

new_bottom3= shift_bottom(n,new_bottom2)
new_bottom3
(top*new_bottom3).order() #has order 1

new_bottom4= shift_bottom(n,new_bottom3)
new_bottom4
(top*new_bottom4).order() #has order 3 







In [ ]:
n=6
Sn = SymmetricGroup(n)
top = Sn('(1,2)(3,6)(4,5)') 
bottom = Sn('(1,4)(2,3)(5,6)')
top * bottom
(top * bottom).order()
new_bottom=shift_bottom(n,bottom)
new_bottom
top*new_bottom
(top*new_bottom).order()





NameError: name 'SymmetricGroup' is not defined

In [ ]:
#the order flip flops here, i dont know why 
n=6
Sn = SymmetricGroup(n)
top = Sn('(1,2)(3,6)(4,5)') 
bottom = Sn('(1,6)(3,4)(2,5)')
top * bottom
(top * bottom).order() #has order 3

new_bottom=shift_bottom(n,bottom)
new_bottom 
top*new_bottom #idk why this wont reurn a perk
(top*new_bottom).order() #has order 1

new_bottom2= shift_bottom(n,new_bottom)
(top*new_bottom2).order() #has order 3 now 

new_bottom3= shift_bottom(n,new_bottom2)
new_bottom3
(top*new_bottom3).order() #has order 3

new_bottom4=shift_bottom(n,new_bottom3)
new_bottom4
(top*new_bottom4).order() #has order 1

new_bottom5=shift_bottom(n,new_bottom4)
new_bottom5
(top*new_bottom5).order() #order 3 



In [ ]:
([(1, 4), (3, 2), (5, 6)], [(1, 6), (3, 4), (5, 2)])
n=6
Sn = SymmetricGroup(n)
top = Sn('(1,4)(2,3)(5,6)') 
bottom = Sn('(1,6)(3,4)(2,5)')
top * bottom
(top * bottom).order()
new_bottom=shift_bottom(n,bottom)
new_bottom 
top*new_bottom #idk why this wont reurn a perk
(top*new_bottom).order()
new_bottom2= shift_bottom(n,new_bottom)

new_bottom2
(top*new_bottom2).order() #has order 1 now the shift is causing the dot maps to disconnect and this is now not an exmaple 

new_bottom3= shift_bottom(n,new_bottom2)
new_bottom3
(top*new_bottom3).order() #has order 3

new_bottom4=shift_bottom(n,new_bottom3)
new_bottom4
(top*new_bottom4).order() #has order 3

NameError: name 'SymmetricGroup' is not defined

In [ ]:
([(1, 4), (3, 6), (5, 2)], [(1, 6), (3, 2), (5, 4)])
#the last line of the valid perms and this example im doing the shifting 
n=6
Sn = SymmetricGroup(n)
top = Sn('(1,4)(3,6)(2,5)') 
bottom = Sn('(1,6)(2,3)(4,5)')
top * bottom
(top * bottom).order()
new_bottom=shift_bottom(n,bottom)
new_bottom 
top*new_bottom 
(top*new_bottom).order()
#shifting again
new_bottom2= shift_bottom(n,new_bottom)
new_bottom2
(top*new_bottom2).order()

new_bottom3= shift_bottom(n,new_bottom2)
new_bottom3
(top*new_bottom3).order() #3rd shift order is the same 

new_bottom4= shift_bottom(n,new_bottom3)
new_bottom4
(top*new_bottom4).order() #order 3

#we want that it stays consistent at order 3, in order to get through all 6 the combo needs to have order 3 



In [ ]:
n=6
Sn = SymmetricGroup(n)
top = Sn('(1,4)(3,6)(2,5)') 
bottom = Sn('(1,6)(2,3)(4,5)')

pillowcase = build_pillowcase(n, top, bottom)
pillowcase

new_bottom= shift_bottom(6,bottom)
new_bottom
new_pillowcase= build_pillowcase(n, top, new_bottom)
new_pillowcase

U = new_pillowcase.minimal_cover("translation") 
U.stratum()
from flatsurf import GL2ROrbitClosure
O = GL2ROrbitClosure(U)  # optional: pyflatsurf
O.dimension() 







In [ ]:
###This is the first n=8 example, the list of valid perms is way too long. WHat i need to do is create a function that takes this list of valid perms at n =8 and checks the order every time i shift the perm
# we already defined the shifting function. i think i need to define an order function 

#i expect perm to be (top,bottom) where top= Sn('(1,2)(3,4)(5,6)') and bottom= Sn('(1,4)(3,6)(2,5)')
def permutation_order(n,perm):
    (top,bottom) = perm
    # print(top)
    # print(bottom)
    before_shift= (top*bottom).order()
    # print(before_shift)

    old_bottom = bottom
    for i in range(n): #this will make the for loop happen n times even tho were not using i
        new_bottom = shift_bottom(n, old_bottom)
        order= (top*new_bottom).order()
        if order==n/2:
            old_bottom = new_bottom
            continue
        else:
            return False # this is how we say NO
    return True 
    
def check_all_perms(n,valid_perms):
    perm_order=[]
    for p in valid_perms:   # p is a variab;le that represents an element of the list "elements"..[a,b,c] first loop=a second loop=b

        if permutation_order(n,p)==True:
            perm_order.append(p) #a dot means calling a function , p is the variable that is equaling each valid perm thats why it goes there
        else: 
            continue 
    return perm_order #return your new list

    

#i need something that calculates the order after the shifts





In [ ]:
from sage.all import SymmetricGroup
#i want to output the list of valid permuatations for n=8 
n = 10
Sn = SymmetricGroup(n)

set = list(range(1, n+1))
valid_perms= compute_valid_perms(set)
valid_pairs= compute_valid_pairs(n, valid_perms)
print(valid_pairs)
full_order_pairs = check_all_perms(n,valid_pairs) #full order is where take pair of top bottom and check order it stayed consistent 
full_order_pairs
#len(pass in a list for the length)


[((1,2)(3,4)(5,6)(7,8)(9,10), (1,4)(2,3)(5,8)(6,9)(7,10)), ((1,2)(3,4)(5,6)(7,8)(9,10), (1,4)(2,3)(5,10)(6,7)(8,9)), ((1,2)(3,4)(5,6)(7,8)(9,10), (1,4)(2,5)(3,6)(7,10)(8,9)), ((1,2)(3,4)(5,6)(7,8)(9,10), (1,4)(2,9)(3,6)(5,8)(7,10)), ((1,2)(3,4)(5,6)(7,8)(9,10), (1,4)(2,7)(3,6)(5,10)(8,9)), ((1,2)(3,4)(5,6)(7,8)(9,10), (1,4)(2,5)(3,8)(6,9)(7,10)), ((1,2)(3,4)(5,6)(7,8)(9,10), (1,4)(2,7)(3,8)(5,10)(6,9)), ((1,2)(3,4)(5,6)(7,8)(9,10), (1,4)(2,9)(3,8)(5,10)(6,7)), ((1,2)(3,4)(5,6)(7,8)(9,10), (1,4)(2,5)(3,10)(6,7)(8,9)), ((1,2)(3,4)(5,6)(7,8)(9,10), (1,4)(2,7)(3,10)(5,8)(6,9)), ((1,2)(3,4)(5,6)(7,8)(9,10), (1,4)(2,9)(3,10)(5,8)(6,7)), ((1,2)(3,4)(5,6)(7,8)(9,10), (1,6)(2,3)(4,5)(7,10)(8,9)), ((1,2)(3,4)(5,6)(7,8)(9,10), (1,6)(2,3)(4,9)(5,8)(7,10)), ((1,2)(3,4)(5,6)(7,8)(9,10), (1,6)(2,3)(4,7)(5,10)(8,9)), ((1,2)(3,4)(5,6)(7,8)(9,10), (1,6)(2,5)(3,8)(4,9)(7,10)), ((1,2)(3,4)(5,6)(7,8)(9,10), (1,6)(2,9)(3,8)(4,5)(7,10)), ((1,2)(3,4)(5,6)(7,8)(9,10), (1,6)(2,7)(3,8)(4,9)(5,10)), ((1,2)(3,4)(5

[((1,2)(3,4)(5,6)(7,8)(9,10), (1,4)(2,9)(3,6)(5,8)(7,10)),
 ((1,2)(3,4)(5,6)(7,8)(9,10), (1,4)(2,5)(3,8)(6,9)(7,10)),
 ((1,2)(3,4)(5,6)(7,8)(9,10), (1,4)(2,7)(3,10)(5,8)(6,9)),
 ((1,2)(3,4)(5,6)(7,8)(9,10), (1,6)(2,7)(3,8)(4,9)(5,10)),
 ((1,2)(3,4)(5,6)(7,8)(9,10), (1,6)(2,9)(3,10)(4,7)(5,8)),
 ((1,2)(3,4)(5,6)(7,8)(9,10), (1,8)(2,5)(3,6)(4,9)(7,10)),
 ((1,2)(3,4)(5,6)(7,8)(9,10), (1,8)(2,9)(3,6)(4,7)(5,10)),
 ((1,2)(3,4)(5,6)(7,8)(9,10), (1,8)(2,5)(3,10)(4,7)(6,9)),
 ((1,2)(3,4)(5,8)(6,9)(7,10), (1,6)(2,7)(3,8)(4,9)(5,10)),
 ((1,2)(3,4)(5,10)(6,7)(8,9), (1,4)(2,9)(3,6)(5,8)(7,10)),
 ((1,2)(3,4)(5,10)(6,7)(8,9), (1,8)(2,5)(3,10)(4,7)(6,9)),
 ((1,2)(3,6)(4,5)(7,10)(8,9), (1,6)(2,7)(3,8)(4,9)(5,10)),
 ((1,2)(3,6)(4,7)(5,8)(9,10), (1,6)(2,7)(3,8)(4,9)(5,10)),
 ((1,2)(3,8)(4,5)(6,7)(9,10), (1,4)(2,9)(3,6)(5,8)(7,10)),
 ((1,2)(3,8)(4,5)(6,7)(9,10), (1,8)(2,5)(3,10)(4,7)(6,9)),
 ((1,2)(3,10)(4,5)(6,9)(7,8), (1,6)(2,7)(3,8)(4,9)(5,10)),
 ((1,2)(3,10)(4,7)(5,6)(8,9), (1,6)(2,7)(3,8)(4,9)(5,10)

In [ ]:
from surface_dynamics.all import Origami
#the holonomy double cover 
#Pauls interesting example 
right = '(1,2,3,4,5,6,7,8,9,10)(20,19,18,17,16,15,14,13,12,11)'
up= '(1,12,9,20,7,16,3,18,5,14)(2,11,4,15,8,13,6,17,10,19)'
o=Origami(right,up)
t = o.teichmueller_curve()
g= o.veech_group()
g.ncusps()
# cusps= t.cusp_representatives()
# c= cusps[0][0] #first zero is first thing in list, second zero is first thing in the tuple 
# c.cylinder_diagram()



5

In [ ]:
#trying this for the n=6 example 
right = '(1,2,3,4,5,6)(12,11,10,9,8,7)'
up= '(1,8,5,12,3,10)(2,7,4,9,6,11)'
# right= '(1)'
# up='(1)'
o=Origami(right,up)
# # t = o.teichmueller_curve()
g= o.veech_group()
o.genus()
g.ncusps() 
# cusps= t.cusp_representatives()
# c= cusps[0][0] #first zero is first thing in list, second zero is first thing in the tuple 
# c.cylinder_diagram()

NameError: name 'Origami' is not defined

In [ ]:
#Second interesting example perm 
right = '(1,2,3,4,5,6,7,8,9,10)(20,19,18,17,16,15,14,13,12,11)'
up= '(1,14,9,18,3,20,5,12,7,16)(2,15,10,13,8,19,4,11,6,17)'
o= Origami(right,up)
g= o.veech_group()
o.genus()
g.ncusps()
T= o.teichmueller_curve()
T.cusp_representatives()
# T.cylinder_diagram()
# o.cylinder_diagram()



[((1,2,3,4,5,6,7,8,9,10)(11,12,13,14,15,16,17,18,19,20)
  (1,11,7,19,9,13,5,15,3,17)(2,12,6,20,8,18,10,14,4,16),
  1),
 ((1,2)(3,4)(5,6,7,8,9,10)(11,12)(13,14,15,16,17,18)(19,20)
  (1,3,5,11,7,16,20,18,9,14)(2,13,19,12,10,15,6,4,8,17),
  3),
 ((1,2,3,4,5,6,7,8,9,10)(11,12,13,14,15,16,17,18,19,20)
  (1,11,5,17,9,19)(2,12,4,14,6,18)(3,15)(7,13)(8,20)(10,16),
  5)]

In [129]:
#automating to compute veech n stuff 
#we write a function that will take in a top,bottom pair and outputs the origami 
# top and bottom are SymmetricGroupElements
def get_double_cover(n,pair):
    (top,bottom)= pair
    up = []
    next= 1
    i=0 
    prime= False 
    while (i<n):
        current= next
        if prime: 
            next= bottom(current-n) 
        else:
            next= top(current)+n
        up.append((current,next))
        prime= not prime
        i+=1 #i = i +1
    
    next= 2
    i=0 
    prime= False 
    while (i<n):
        current= next
        if prime: 
            next= bottom(current-n) 
        else:
            next= top(current)+n
        up.append((current,next))
        prime= not prime
        i+=1 #i = i +1

    s = ''
    for i, pair in enumerate(up): 
        
        # start of perm 
        if s == '' or s[-1] == ')':
            s = s + f'({pair[0]},'
        # end of perm overall
        elif i == len(up)-1:
            s = s + f'{pair[0]})'
        # end of perm in middle
        elif up[i+1][0] != pair[1]:
            s = s + f'{pair[0]})('
        # middle of perm
        else:
            s = s + f'{pair[0]},'
    up = s
    
    # compute right
    right = ''
    for i in range(1, n+1):
        if i == 1:
            right = right+'(1,'
        elif i < n:
            right = right + f'{i},'
        if i == n:
            right = right + f'{n})'

    for i in range(n):
        if i == 0:
            right = right+f'({n+n},'
        elif i < n-1:
            right = right + f'{n-i+n},'
        else:
            right = right + f'{n-i+n})'

    return Origami(right,up)




        



In [146]:
from sage.all import SymmetricGroup
n = 10
Sn = SymmetricGroup(n)

set = list(range(1, n+1))
valid_perms= compute_valid_perms(set)
valid_pairs= compute_valid_pairs(n, valid_perms)
full_order_pairs = check_all_perms(n,valid_pairs) #full order is where take pair of top bottom and check order it stayed consistent 


#going through each of the n=10 examples for the number of cusps and number of cyls 
#interesting examples 
one_cyl= []
for pair in full_order_pairs: 
    o = get_double_cover(n, pair)
    type(o)
    T=o.teichmueller_curve()
    cylinder_count(T)
    if cylinder_count(T)==True:
        one_cyl.append(pair)
    else:
        continue 
len(one_cyl)
print(one_cyl)



[((1,2)(3,4)(5,6)(7,8)(9,10), (1,4)(2,9)(3,6)(5,8)(7,10)), ((1,2)(3,4)(5,6)(7,8)(9,10), (1,4)(2,5)(3,8)(6,9)(7,10)), ((1,2)(3,4)(5,6)(7,8)(9,10), (1,4)(2,7)(3,10)(5,8)(6,9)), ((1,2)(3,4)(5,6)(7,8)(9,10), (1,6)(2,7)(3,8)(4,9)(5,10)), ((1,2)(3,4)(5,6)(7,8)(9,10), (1,6)(2,9)(3,10)(4,7)(5,8)), ((1,2)(3,4)(5,6)(7,8)(9,10), (1,8)(2,5)(3,6)(4,9)(7,10)), ((1,2)(3,4)(5,6)(7,8)(9,10), (1,8)(2,9)(3,6)(4,7)(5,10)), ((1,2)(3,4)(5,6)(7,8)(9,10), (1,8)(2,5)(3,10)(4,7)(6,9)), ((1,4)(2,9)(3,6)(5,8)(7,10), (1,6)(2,7)(3,8)(4,9)(5,10)), ((1,4)(2,9)(3,6)(5,8)(7,10), (1,10)(2,3)(4,5)(6,7)(8,9)), ((1,4)(2,5)(3,8)(6,9)(7,10), (1,10)(2,3)(4,5)(6,7)(8,9)), ((1,4)(2,7)(3,10)(5,8)(6,9), (1,10)(2,3)(4,5)(6,7)(8,9)), ((1,6)(2,7)(3,8)(4,9)(5,10), (1,8)(2,5)(3,10)(4,7)(6,9)), ((1,6)(2,7)(3,8)(4,9)(5,10), (1,10)(2,3)(4,5)(6,7)(8,9)), ((1,6)(2,9)(3,10)(4,7)(5,8), (1,10)(2,3)(4,5)(6,7)(8,9)), ((1,8)(2,5)(3,6)(4,9)(7,10), (1,10)(2,3)(4,5)(6,7)(8,9)), ((1,8)(2,9)(3,6)(4,7)(5,10), (1,10)(2,3)(4,5)(6,7)(8,9)), ((1,8)(2,5)(3

In [ ]:
example_one= ('(1,2,3,4,5,6,7,8,9,10)(20,19,18,17,16,15,14,13,12,11)'
,'(1,12,9,20,7,16,3,18,5,14)(2,11,4,15,8,13,6,17,10,19)')

example_two= ('(1,2,3,4,5,6,7,8,9,10)(20,19,18,17,16,15,14,13,12,11)'
,'(1,14,9,18,3,20,5,12,7,16)(2,15,10,13,8,19,4,11,6,17)')

examples= [example_one,example_two]
for example in examples:
    o=Origami(example[0],example[1])
    T=o.teichmueller_curve()
    cylinder_count(T)
    


False
T has 5 cusps and 2 of them have 2 cylinders
False
T has 3 cusps and 2 of them have 2 cylinders


In [145]:
#checking for the interesting example from class 
#T is a teichmueller curve 
def cylinder_count(T):
    counter1 = 0
    counter2 = 0
    for c in T.cusp_representatives():
        counter1 = counter1 + 1
        cd = c[0].cylinder_diagram()
        num = cd.ncyls()
        if num == 2:
            counter2 = counter2 + 1
    # print(f'T has {counter1} cusps and {counter2} of them have 2 cylinders')
    if counter1 == counter2:
        return True
    else: 
        return False 